In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt


In [12]:


def gen_momentum_vectors(N):
    #generate random 2d, unit vectors ---> (x, z)
    x_vals = np.random.uniform(-1, 1, size = N)
    z_vals = np.random.uniform(-1, 1, size = N)
    
    J_plus = np.asarray([np.asarray([x, z]).reshape(2,1) for x, z in zip(x_vals, z_vals)])
    J_minus = np.asarray([-1*Jp for Jp in J_plus])
    
    return [J_plus, J_minus]


vecs = gen_momentum_vectors(10)


vecs[1]    
    
    
    







array([[[-0.39502022],
        [-0.4334003 ]],

       [[ 0.32482916],
        [-0.6730454 ]],

       [[ 0.51031133],
        [ 0.20559264]],

       [[ 0.861319  ],
        [ 0.06702416]],

       [[-0.3704702 ],
        [-0.39766989]],

       [[-0.75079127],
        [-0.68463988]],

       [[-0.8177924 ],
        [ 0.87009409]],

       [[-0.61985342],
        [-0.73343526]],

       [[ 0.5737489 ],
        [ 0.83681038]],

       [[ 0.39345755],
        [-0.86525328]]])

In [13]:
vecs[0]

array([[[ 0.39502022],
        [ 0.4334003 ]],

       [[-0.32482916],
        [ 0.6730454 ]],

       [[-0.51031133],
        [-0.20559264]],

       [[-0.861319  ],
        [-0.06702416]],

       [[ 0.3704702 ],
        [ 0.39766989]],

       [[ 0.75079127],
        [ 0.68463988]],

       [[ 0.8177924 ],
        [-0.87009409]],

       [[ 0.61985342],
        [ 0.73343526]],

       [[-0.5737489 ],
        [-0.83681038]],

       [[-0.39345755],
        [ 0.86525328]]])